In [111]:
import os
import ast
import numpy as np 
import pandas as pd

In [1]:
# import wyscout public match event data loader from socceraction library
from socceraction.data.statsbomb import StatsBombLoader 

# remove credentials warning from statsbomb api due to public data access 
import warnings
warnings.filterwarnings("ignore", message="credentials were not supplied. open data access only")


# load public wyscout data
stbm_data = StatsBombLoader()

In [12]:
competitions = stbm_data.competitions()
female_comps = competitions.loc[competitions['competition_gender'] == 'female', :].reset_index(drop = True)
female_comps

,season_id,competition_id,competition_name,country_name,competition_gender,season_name
0,90,37,FA Women's Super League,England,female,2020/2021
1,42,37,FA Women's Super League,England,female,2019/2020
2,4,37,FA Women's Super League,England,female,2018/2019
3,3,49,NWSL,United States of America,female,2018
4,106,53,UEFA Women's Euro,Europe,female,2022
5,30,72,Women's World Cup,International,female,2019


In [47]:
dir_names = ['FAWSL_2021', 'FAWSL_1920', 'FAWSL_1819', 'NWSL', 'EURO_2022', 'WC_2019']

n_games = 0
for i, j in female_comps.loc[:, ['season_id', 'competition_id']].iterrows():
    # j[0] = season_id, j[1] = competition_id
    games = stbm_data.games(j[1], j[0]).loc[:, 'game_id']
    for k in games:
        print(k)
        
        location, related_events, extra
        
        events = stbm_data.events(k)
        events.to_csv(f'data/{dir_names[i]}/games/{k}.csv', index = False)

In [48]:
all_events = pd.DataFrame()
for i in dir_names:
    games = os.listdir(f'data/{i}/games')
    for j in games:
        df = pd.read_csv(f'data/{i}/games/{j}')
        all_events = pd.concat([all_events, df])

In [68]:
all_events = all_events.reset_index(drop = True)
all_events.to_csv('data/all_events.csv', index = False)

In [69]:
all_events.columns

Index(['game_id', 'event_id', 'period_id', 'team_id', 'player_id', 'type_id',
       'type_name', 'index', 'timestamp', 'minute', 'second', 'possession',
       'possession_team_id', 'possession_team_name', 'play_pattern_id',
       'play_pattern_name', 'team_name', 'duration', 'extra', 'related_events',
       'player_name', 'position_id', 'position_name', 'location',
       'under_pressure', 'counterpress'],
      dtype='object')

In [101]:
shots = all_events.loc[all_events['type_name'] == 'Shot', ['type_name', 'play_pattern_name', 'position_name', 
                                                           'location', 'under_pressure', 'extra']].reset_index(drop = True)

In [227]:
shots.loc[:, 'extra'] = shots.loc[:, 'extra'].apply(ast.literal_eval)

In [228]:
shots.head()

,type_name,play_pattern_name,position_name,location,under_pressure,extra
0,Shot,Regular Play,Right Wing,"[95.9, 58.9]",False,"{'shot': {'statsbomb_xg': 0.013642391, 'end_lo..."
1,Shot,From Free Kick,Center Forward,"[106.1, 54.3]",False,"{'shot': {'statsbomb_xg': 0.04084396, 'end_loc..."
2,Shot,From Free Kick,Left Wing,"[110.0, 28.2]",True,"{'shot': {'statsbomb_xg': 0.13687119, 'end_loc..."
3,Shot,From Throw In,Center Attacking Midfield,"[113.2, 40.4]",False,"{'shot': {'statsbomb_xg': 0.12462413, 'end_loc..."
4,Shot,From Counter,Center Attacking Midfield,"[95.2, 39.8]",False,"{'shot': {'statsbomb_xg': 0.02380701, 'end_loc..."


In [232]:
for i in shots.loc[:, 'extra'][:10]:
    print(i.values())

dict_values([{'statsbomb_xg': 0.013642391, 'end_location': [116.7, 44.9, 1.2], 'key_pass_id': 'cbd54df9-6804-4fe1-96d5-4891d7d2f5ba', 'type': {'id': 87, 'name': 'Open Play'}, 'outcome': {'id': 100, 'name': 'Saved'}, 'deflected': True, 'technique': {'id': 93, 'name': 'Normal'}, 'body_part': {'id': 38, 'name': 'Left Foot'}, 'freeze_frame': [{'location': [119.6, 42.3], 'player': {'id': 42787, 'name': 'Mackenzie Arnold'}, 'position': {'id': 1, 'name': 'Goalkeeper'}, 'teammate': False}, {'location': [107.1, 36.2], 'player': {'id': 31550, 'name': 'Angela Addison'}, 'position': {'id': 21, 'name': 'Left Wing'}, 'teammate': True}, {'location': [107.5, 38.9], 'player': {'id': 34232, 'name': 'Grace Fisk'}, 'position': {'id': 2, 'name': 'Right Back'}, 'teammate': False}, {'location': [100.8, 57.1], 'player': {'id': 15724, 'name': 'Mayumi Pacheco'}, 'position': {'id': 6, 'name': 'Left Back'}, 'teammate': False}, {'location': [105.7, 61.0], 'player': {'id': 31556, 'name': 'Laura Vetterlein'}, 'posit

In [247]:
item

{'shot': {'statsbomb_xg': 0.013642391,
  'end_location': [116.7, 44.9, 1.2],
  'key_pass_id': 'cbd54df9-6804-4fe1-96d5-4891d7d2f5ba',
  'type': {'id': 87, 'name': 'Open Play'},
  'outcome': {'id': 100, 'name': 'Saved'},
  'deflected': True,
  'technique': {'id': 93, 'name': 'Normal'},
  'body_part': {'id': 38, 'name': 'Left Foot'},
  'freeze_frame': [{'location': [119.6, 42.3],
    'player': {'id': 42787, 'name': 'Mackenzie Arnold'},
    'position': {'id': 1, 'name': 'Goalkeeper'},
    'teammate': False},
   {'location': [107.1, 36.2],
    'player': {'id': 31550, 'name': 'Angela Addison'},
    'position': {'id': 21, 'name': 'Left Wing'},
    'teammate': True},
   {'location': [107.5, 38.9],
    'player': {'id': 34232, 'name': 'Grace Fisk'},
    'position': {'id': 2, 'name': 'Right Back'},
    'teammate': False},
   {'location': [100.8, 57.1],
    'player': {'id': 15724, 'name': 'Mayumi Pacheco'},
    'position': {'id': 6, 'name': 'Left Back'},
    'teammate': False},
   {'location': [1

In [297]:
shotsframe = shots.loc[:, 'extra'].reset_index(drop = True)

In [308]:
keys = ['follows_dribble', 'first_time', 'open_goal', 'technique', 'body_part', 'type']
#features = {'follows_dribble':[], 'first_time':[], 'open_goal':[], 'technique':[], 'body_part':[], 'type':[]}
features = pd.DataFrame(np.nan, columns = keys, index = range(shots.shape[0]))
for i, j in shotsframe.iteritems():
    for k in list(j['shot'].keys()):
        if k in ['type', 'technique', 'body_part']:
            features.loc[i, k] = j['shot'][k]['name']
        elif k in keys:
            #features[k].append(j['shot'][k])
            features.loc[i, k] = j['shot'][k]

In [311]:
features.head()

,follows_dribble,first_time,open_goal,technique,body_part,type
0,NaN,NaN,NaN,Normal,Left Foot,Open Play
1,NaN,NaN,NaN,Normal,Right Foot,Open Play
2,NaN,NaN,NaN,Normal,Left Foot,Open Play
3,NaN,NaN,NaN,Normal,Head,Open Play
4,NaN,NaN,NaN,Normal,Left Foot,Open Play
